### 0/1 Classification

In [2]:
import os
import numpy as np
os.chdir("/content")
!unzip btype==1.zip
!unzip btype==2.zip
!unzip btype==4.zip

Streaming output truncated to the last 5000 lines.
  inflating: btype==2/904 1726.npy   
  inflating: btype==2/904 1727.npy   
  inflating: btype==2/904 1728.npy   
  inflating: btype==2/904 1729.npy   
  inflating: btype==2/904 1730.npy   
  inflating: btype==2/904 1731.npy   
  inflating: btype==2/904 1732.npy   
  inflating: btype==2/904 1733.npy   
  inflating: btype==2/904 1734.npy   
  inflating: btype==2/904 1735.npy   
  inflating: btype==2/904 1736.npy   
  inflating: btype==2/904 1737.npy   
  inflating: btype==2/904 1738.npy   
  inflating: btype==2/904 1739.npy   
  inflating: btype==2/904 1740.npy   
  inflating: btype==2/904 1741.npy   
  inflating: btype==2/904 1742.npy   
  inflating: btype==2/904 1743.npy   
  inflating: btype==2/904 1744.npy   
  inflating: btype==2/904 1745.npy   
  inflating: btype==2/906 1747.npy   
  inflating: btype==2/906 1748.npy   
  inflating: btype==2/907 1749.npy   
  inflating: btype==2/907 1750.npy   
  inflating: btype==2/907 1751.npy   

In [4]:
arr1=np.array([])
import os
for files in os.listdir("/content/btype==1"):
  os.chdir("/content/btype==1")
  file_name=os.path.abspath(files)
  new_read=np.load(file_name,allow_pickle=True)  
  new_read=np.append(new_read,[0])
  new_read=np.transpose(new_read)
  arr1=np.append(arr1,new_read,axis=-1)
print(arr1.shape)

arr2=np.array([])
for files in os.listdir("/content/btype==2"):
  os.chdir("/content/btype==2")
  file_name=os.path.abspath(files)
  new_read=np.load(file_name,allow_pickle=True)
  if(len(new_read)!=2048):
    continue  
  new_read=np.append(new_read,[1])
  new_read=np.transpose(new_read)
  arr2=np.append(arr2,new_read,axis=-1)
print(arr2.shape)


arr3=np.array([])
for files in os.listdir("/content/btype==4"):
  os.chdir("/content/btype==4")
  file_name=os.path.abspath(files)
  new_read=np.load(file_name,allow_pickle=True) 
  if(len(new_read)!=2048):
    continue 
  new_read=np.append(new_read,[1])
  new_read=np.transpose(new_read)
  arr3=np.append(arr3,new_read,axis=-1)
print(arr3.shape)




(12007140,)
(10527762,)
(9968385,)


In [5]:
def chunks(l, n):
    return [l[i:i+n] for i in range(0, len(l), n)]
labels_1=chunks(arr1,2049)
labels_2=chunks(arr2,2049)
labels_3=chunks(arr3,2049)
labels_1=np.array(labels_1)
labels_2=np.array(labels_2)
labels_3=np.array(labels_3)
labels=np.concatenate((labels_1,labels_2,labels_3))

In [6]:
np.random.shuffle(labels)
np.random.shuffle(labels)
end_1=len(labels)//100
end_1=end_1*75
end_2=(len(labels))//100
end_2=end_2*90
train,val,test=labels[:end_1],labels[end_1:end_2],labels[end_2:]

In [7]:
from keras.utils.np_utils import to_categorical
X_train=train[:,:2048]
print(X_train.shape)
Y_train=train[:,2048]
print(Y_train.shape)
X_val=val[:,:2048]
print(X_val.shape)
Y_val=val[:,2048]
print(Y_val.shape)
X_test=test[:,:2048]
print(X_test.shape)
Y_test=test[:,2048]
print(Y_test.shape)
Y_train=to_categorical(Y_train)
Y_val=to_categorical(Y_val)
Y_test=to_categorical(Y_test)


(11850, 2048)
(11850,)
(2370, 2048)
(2370,)
(1643, 2048)
(1643,)


In [8]:
!pip install keras
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers import BatchNormalization
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
def network(X_train,y_train,X_val,y_val):
    im_shape=(X_train.shape[1],1)
    inputs_cnn=Input(shape=(im_shape), name='inputs_cnn')
    conv1_1=Convolution1D(64, (6), activation='relu', input_shape=im_shape)(inputs_cnn)
    conv1_1=BatchNormalization()(conv1_1)
    pool1=MaxPool1D(pool_size=(3), strides=(2), padding="same")(conv1_1)
    conv2_1=Convolution1D(64, (3), activation='relu', input_shape=im_shape)(pool1)
    conv2_1=BatchNormalization()(conv2_1)
    pool2=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv2_1)
    conv3_1=Convolution1D(64, (3), activation='relu', input_shape=im_shape)(pool2)
    conv3_1=BatchNormalization()(conv3_1)
    pool3=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv3_1)
    flatten=Flatten()(pool3)
    dense_end1 = Dense(64, activation='relu')(flatten)
    dense_end2 = Dense(32, activation='relu')(dense_end1)
    main_output = Dense(2, activation='softmax', name='main_output')(dense_end2)
    
    
    model = Model(inputs= inputs_cnn, outputs=main_output)
    model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics = ['accuracy'])
    model.summary()
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

    history=model.fit(X_train, y_train,epochs=40,callbacks=callbacks, batch_size=32,validation_data=(X_val,y_val))
    # history=model.fit(X_train, y_train,epochs=25, batch_size=32,validation_data=(X_val,y_val))
    model.load_weights('best_model.h5')
    return(model,history)

os.chdir("/content")
model,history=network(X_train,Y_train,X_val,Y_val)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs_cnn (InputLayer)     [(None, 2048, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2043, 64)          448       
                                                                 
 batch_normalization (BatchN  (None, 2043, 64)         256       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1022, 64)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1020, 64)          12352     
                                                                 
 batch_normalization_1 (Batc  (None, 1020, 64)         256   

In [9]:
from keras.models import load_model
os.chdir("/content")
loaded=load_model("best_model.h5")
pred=model.predict(X_test)

In [57]:
y_pred=[]
for i in range(1643):
  if(np.argmax(pred[i])==1):
    y_pred.append(1)
  else:
    y_pred.append(0)
y_act=[]
for i in range(1643):
  if(test[i][2048]==1):
    y_act.append(1)
  else:
    y_act.append(0)

In [19]:
from sklearn import metrics
print(metrics.confusion_matrix(y_act, y_pred, labels=[0,1]))
print(metrics.classification_report(y_act, y_pred, labels=[0,1]))

[[510 108]
 [ 95 930]]
              precision    recall  f1-score   support

           0       0.84      0.83      0.83       618
           1       0.90      0.91      0.90      1025

    accuracy                           0.88      1643
   macro avg       0.87      0.87      0.87      1643
weighted avg       0.88      0.88      0.88      1643



In [ ]:
!pip install biosppy
import biosppy
def r_peaks(signals):
  peaks=biosppy.signals.ecg.christov_segmenter(signal=signals, sampling_rate=250)
  return peaks
def anamoly_RR_interval(signals):
  peaks=r_peaks(signals)
  count=0
  peaks=np.array(peaks)
  intervals=np.diff(peaks)
  for i in range(len(intervals)):
    if(intervals[i]>300 or intervals[i]<150):
      count+=1
  if(count>(len(peaks)//4)):
    return 1
  return 0


In [65]:
for i in range(1):
    sig=X_test[i]
    peaks=biosppy.signals.ecg.christov_segmenter(signal=sig, sampling_rate=250)
    count=0
    peaks=np.array(peaks)
    intervals=np.diff(peaks)
    for i in range(len(intervals[0])):
      if(intervals[0][i]>300 or intervals[0][i]<150):
        count+=1
    if(count):
      y_pred[i]=1

In [55]:
from sklearn.metrics import classification_report
print(classification_report(y_act, y_pred, target_names=['0','1']))

              precision    recall  f1-score   support

           0       0.84      0.83      0.83       618
           1       0.90      0.91      0.90      1025

    accuracy                           0.88      1643
   macro avg       0.87      0.87      0.87      1643
weighted avg       0.88      0.88      0.88      1643



In [23]:
from sklearn.metrics import classification_report
print(classification_report(y_act, y_pred, target_names=['0','1']))

              precision    recall  f1-score   support

           0       0.84      0.83      0.83       618
           1       0.90      0.91      0.90      1025

    accuracy                           0.88      1643
   macro avg       0.87      0.87      0.87      1643
weighted avg       0.88      0.88      0.88      1643



### Class Classification

0-Normal(1)
1-PAC(2)
2-PVC(4)



In [66]:
arr1=np.array([])
import os
for files in os.listdir("/content/btype==1"):
  os.chdir("/content/btype==1")
  file_name=os.path.abspath(files)
  new_read=np.load(file_name,allow_pickle=True)  
  new_read=np.append(new_read,[0])
  new_read=np.transpose(new_read)
  arr1=np.append(arr1,new_read,axis=-1)
print(arr1.shape)

arr2=np.array([])
for files in os.listdir("/content/btype==2"):
  os.chdir("/content/btype==2")
  file_name=os.path.abspath(files)
  new_read=np.load(file_name,allow_pickle=True)
  if(len(new_read)!=2048):
    continue  
  new_read=np.append(new_read,[1])
  new_read=np.transpose(new_read)
  arr2=np.append(arr2,new_read,axis=-1)
print(arr2.shape)

arr3=np.array([])
for files in os.listdir("/content/btype==4"):
  os.chdir("/content/btype==4")
  file_name=os.path.abspath(files)
  new_read=np.load(file_name,allow_pickle=True) 
  if(len(new_read)!=2048):
    continue 
  new_read=np.append(new_read,[2])
  new_read=np.transpose(new_read)
  arr3=np.append(arr3,new_read,axis=-1)
print(arr3.shape)



(12007140,)
(10527762,)
(9968385,)


In [67]:
def chunks(l, n):
    return [l[i:i+n] for i in range(0, len(l), n)]
labels_1=chunks(arr1,2049)
labels_2=chunks(arr2,2049)
labels_3=chunks(arr3,2049)
labels_1=np.array(labels_1)
labels_2=np.array(labels_2)
labels_3=np.array(labels_3)
labels=np.concatenate((labels_1,labels_2,labels_3))
np.random.shuffle(labels)
np.random.shuffle(labels)
end_1=len(labels)//100
end_1=end_1*75
end_2=(len(labels))//100
end_2=end_2*90
train,val,test=labels[:end_1],labels[end_1:end_2],labels[end_2:]

In [68]:
from keras.utils.np_utils import to_categorical
X_train=train[:,:2048]
print(X_train.shape)
Y_train=train[:,2048]
print(Y_train.shape)
X_val=val[:,:2048]
print(X_val.shape)
Y_val=val[:,2048]
print(Y_val.shape)
X_test=test[:,:2048]
print(X_test.shape)
Y_test=test[:,2048]
print(Y_test.shape)
Y_train=to_categorical(Y_train)
Y_val=to_categorical(Y_val)
Y_test=to_categorical(Y_test)

(11850, 2048)
(11850,)
(2370, 2048)
(2370,)
(1643, 2048)
(1643,)


In [70]:
!pip install keras
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers import BatchNormalization
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
def network(X_train,y_train,X_val,y_val):
    im_shape=(X_train.shape[1],1)
    inputs_cnn=Input(shape=(im_shape), name='inputs_cnn')
    conv1_1=Convolution1D(64, (6), activation='relu', input_shape=im_shape)(inputs_cnn)
    conv1_1=BatchNormalization()(conv1_1)
    pool1=MaxPool1D(pool_size=(3), strides=(2), padding="same")(conv1_1)
    conv2_1=Convolution1D(64, (3), activation='relu', input_shape=im_shape)(pool1)
    conv2_1=BatchNormalization()(conv2_1)
    pool2=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv2_1)
    conv3_1=Convolution1D(64, (3), activation='relu', input_shape=im_shape)(pool2)
    conv3_1=BatchNormalization()(conv3_1)
    pool3=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv3_1)
    flatten=Flatten()(pool3)
    dense_end1 = Dense(64, activation='relu')(flatten)
    dense_end2 = Dense(32, activation='relu')(dense_end1)
    main_output = Dense(3, activation='softmax', name='main_output')(dense_end2)
    
    
    model = Model(inputs= inputs_cnn, outputs=main_output)
    model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics = ['accuracy'])
    model.summary()
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

    history=model.fit(X_train, y_train,epochs=40,callbacks=callbacks, batch_size=32,validation_data=(X_val,y_val))
    # history=model.fit(X_train, y_train,epochs=25, batch_size=32,validation_data=(X_val,y_val))
    model.load_weights('best_model.h5')
    return(model,history)

os.chdir("/content")
model,history=network(X_train,Y_train,X_val,Y_val)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs_cnn (InputLayer)     [(None, 2048, 1)]         0         
                                                                 
 conv1d_6 (Conv1D)           (None, 2043, 64)          448       
                                                                 
 batch_normalization_6 (Batc  (None, 2043, 64)         256       
 hNormalization)                                                 
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 1022, 64)         0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 1020, 64)          12352     
                                                                 
 batch_normalization_7 (Batc  (None, 1020, 64)         256 

In [72]:
from keras.models import load_model
os.chdir("/content")
loaded=load_model("best_model.h5")
pred=model.predict(X_test)

In [73]:
y_pred=[]
for i in range(1643):
   y_pred.append(np.argmax(pred[i]))


y_act=[]
for i in range(1643):
  y_act.append(test[i][2048])


In [74]:
from sklearn import metrics
print(metrics.confusion_matrix(y_act, y_pred, labels=[0,1,2]))
print(metrics.classification_report(y_act, y_pred, labels=[0,1,2]))

[[485  79  16]
 [ 75 423  50]
 [ 36  84 395]]
              precision    recall  f1-score   support

           0       0.81      0.84      0.82       580
           1       0.72      0.77      0.75       548
           2       0.86      0.77      0.81       515

    accuracy                           0.79      1643
   macro avg       0.80      0.79      0.79      1643
weighted avg       0.80      0.79      0.79      1643

